In [49]:
# Install Requirments (Updated on 12/17/2024)
# !pip3 install -r requirements.txt

In [4]:
import os
import sys

# Suppress TensorFlow logs
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  # Use asynchronous CUDA memory allocator
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set TensorFlow log level
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Restrict TensorFlow to GPU 0
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'  # Prevent CUDA errors

# Redirect STDERR to /dev/null to silence C++ warnings
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_predict

from datetime import datetime

import tensorflow as tf
import logging

# Suppress TensorFlow and absl logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)

# Clear previous sessions
tf.keras.backend.clear_session()

# GPU configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is configured properly.")
    except RuntimeError as e:
        print("Error initializing GPU:", e)
else:
    print("No GPU detected. TensorFlow will use the CPU.")

sys.stderr = stderr  # Restore STDERR

# Check available GPUs
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


GPU is configured properly.
Num GPUs Available: 1


In [3]:
from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [65]:
f.predictions('tsla')

TSLA 5m Interval Timestamp: 2024-12-19 14:27:34 


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,up,up,up,up,static
kelly_1:2.5,0.193118,0.152239,0.193576,0.125,0.064811
prob_up,0.458909,0.537568,0.460139,0.5,0.0
prob_static,0.115763,0.09039,0.122517,0.125,0.0
prob_down,0.425327,0.372042,0.417344,0.375,1.0
precision,0.423656,0.394456,0.423983,0.375,0.332008
recall,0.390873,0.367063,0.392857,0.386905,1.0
f1,0.406605,0.380267,0.407827,0.380859,0.498507
support,"[504.0, 501.0, 504.0]","[504.0, 501.0, 504.0]","[504.0, 501.0, 504.0]","[504.0, 501.0, 504.0]","[504.0, 501.0, 504.0]"


TSLA 15m Interval Timestamp: 2024-12-19 14:27:58 


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,up,up,up,static,static
kelly_1:2.5,0.171264,0.180488,0.198742,0.20524,0.0648
prob_up,0.638183,0.700643,0.571307,0.25,0.0
prob_static,0.077548,0.077378,0.150893,0.5,0.0
prob_down,0.284269,0.221979,0.2778,0.25,1.0
precision,0.408046,0.414634,0.427673,0.432314,0.332
recall,0.42515,0.407186,0.407186,0.596386,1.0
f1,0.416422,0.410876,0.417178,0.501266,0.498498
support,"[167.0, 166.0, 167.0]","[167.0, 166.0, 167.0]","[167.0, 166.0, 167.0]","[167.0, 166.0, 167.0]","[167.0, 166.0, 167.0]"


TSLA 1h Interval Timestamp: 2024-12-19 14:28:07 


...

,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,down,down,down,up,static
kelly_1:2.5,0.085149,0.068868,0.12907,0.085484,0.068611
prob_up,0.362468,0.282467,0.30449,0.375,0.0
prob_static,0.221213,0.12517,0.167545,0.25,0.0
prob_down,0.41632,0.592363,0.527964,0.375,1.0
precision,0.346535,0.334906,0.377907,0.346774,0.334722
recall,0.292887,0.297071,0.271967,0.358333,1.0
f1,0.31746,0.314856,0.316302,0.352459,0.501561
support,"[240.0, 241.0, 239.0]","[240.0, 241.0, 239.0]","[240.0, 241.0, 239.0]","[240.0, 241.0, 239.0]","[240.0, 241.0, 239.0]"


TSLA 1wk Interval Timestamp: 2024-12-19 14:28:47 


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,up,up,up,down,static
kelly_1:2.5,0.15,0.056522,0.25,0.12,0.056944
prob_up,0.733276,0.59913,0.367741,0.25,0.0
prob_static,0.101826,0.125386,0.302164,0.125,0.0
prob_down,0.164897,0.275484,0.330095,0.625,1.0
precision,0.392857,0.326087,0.464286,0.371429,0.326389
recall,0.44898,0.306122,0.530612,0.270833,1.0
f1,0.419048,0.315789,0.495238,0.313253,0.492147
support,"[49.0, 47.0, 48.0]","[49.0, 47.0, 48.0]","[49.0, 47.0, 48.0]","[49.0, 47.0, 48.0]","[49.0, 47.0, 48.0]"


TSLA 1mo Interval Timestamp: 2024-12-19 14:28:52 


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,up,up,up,up,static
kelly_1:2.5,0.125,0.144444,0.144444,0.16,0.02
prob_up,0.932252,0.997903,0.580899,0.5,0.0
prob_static,0.025129,0.000728,0.149315,0.125,0.0
prob_down,0.042619,0.001369,0.269786,0.375,1.0
precision,0.375,0.388889,0.388889,0.4,0.3
recall,0.545455,0.636364,0.636364,0.545455,1.0
f1,0.444444,0.482759,0.482759,0.461538,0.461538
support,"[11.0, 9.0, 10.0]","[11.0, 9.0, 10.0]","[11.0, 9.0, 10.0]","[11.0, 9.0, 10.0]","[11.0, 9.0, 10.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')